In [4]:
import numpy as np
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter

class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))

    def find(self, u):
        if self.parent[u] != u:
            self.parent[u] = self.find(self.parent[u])
        return self.parent[u]

    def union(self, u, v):
        ru, rv = self.find(u), self.find(v)
        if ru != rv:
            self.parent[rv] = ru
            return True
        return False

def generate_boruvka_frames(points):
    n = len(points)
    dist_matrix = squareform(pdist(points))
    uf = UnionFind(n)
    mst_edges = []

    frames = []
    num_components = n

    while num_components > 1:
        cheapest = [None] * n

        for u in range(n):
            for v in range(n):
                if u != v and uf.find(u) != uf.find(v):
                    root = uf.find(u)
                    if cheapest[root] is None or dist_matrix[u][v] < dist_matrix[cheapest[root][0]][cheapest[root][1]]:
                        cheapest[root] = (u, v)

        new_edges = []
        for edge in cheapest:
            if edge is not None:
                u, v = edge
                if uf.find(u) != uf.find(v):
                    new_edges.append((u, v))

        # Armazena o frame: pontos, arestas MST, novas arestas candidatas
        frames.append((points.copy(), mst_edges.copy(), new_edges))

        # Adiciona novas arestas à MST
        for u, v in new_edges:
            if uf.union(u, v):
                mst_edges.append((u, v))
                num_components -= 1

    # Frame final com MST completa
    frames.append((points.copy(), mst_edges.copy(), []))
    return frames

def animate_boruvka(frames, filename="boruvka_mst.mp4"):
    fig, ax = plt.subplots()

    def update(frame):
        points, mst_edges, new_edges = frame
        ax.clear()
        ax.set_title(f"Etapa da MST (arestas: {len(mst_edges)})")
        ax.scatter(points[:, 0], points[:, 1], c='black')

        # Arestas já na MST (azul)
        for u, v in mst_edges:
            ax.plot([points[u][0], points[v][0]], [points[u][1], points[v][1]], 'b-', linewidth=2)

        # Novas arestas candidatas (vermelho tracejado)
        for u, v in new_edges:
            ax.plot([points[u][0], points[v][0]], [points[u][1], points[v][1]], 'r--', linewidth=1.5)

        ax.axis('equal')
        ax.grid(True)

    anim = FuncAnimation(fig, update, frames=frames, interval=1500)

    # Salva como vídeo MP4 (requer ffmpeg instalado)
    writer = FFMpegWriter(fps=1)
    anim.save(filename, writer=writer)
    plt.close(fig)

# Exemplo de uso
if __name__ == "__main__":
    np.random.seed(0)
    points = np.random.rand(20, 2)
    frames = generate_boruvka_frames(points)
    animate_boruvka(frames, "boruvka_mst.mp4")
